## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [10]:
import pandas as pd
import numpy as np
import cassandra
import csv

In [58]:
df = pd.read_csv('event_data.csv')
filtered_df = df[(df['session_number'] == 338) & (df['item_in_session_number'] == 4)]
filtered_df

,artist_name,fname,gender,item_in_session_number,lname,length,level,location,session_number,song_title,user_id
5208,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50



##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [60]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
from cassandra.cluster import Cluster
try:
    cluster=Cluster(['127.0.0.1'],port=9042)
    session = cluster.connect()
    print("Cassendra connected")
except Exception as e:
    print(e)
                     
# create a session to establish connection and begin executing queries


Cassendra connected


#### Create & Set Keyspace

In [61]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above
try:
    session.execute("""
    create keyspace if not exists soundCloud
    with replication ={'class':'SimpleStrategy','replication_factor':1}
    """)
    session.set_keyspace('soundcloud')
    print("connected to Cassendra!")
except Exception as e:
    print(e)

connected to Cassendra!


## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [ ]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command 

# For task 1 Creating a table before inseting data into table

In [28]:
try:
    session.execute("""
            create table if not exists soundCloud.soundCloud_app_history
            (
            session_number int,
            item_in_session_number int,
            artist_name text,
            song_tittle text,
            length float,
            primary key (session_number, item_in_session_number)
            
            )
            """ )
    print("Table soundCloud_app_histroy is created!")


    
except Exception as e:
    print(e)
    
    

Table soundCloud_app_histroy is created!


### Let's insert our data into of table

In [38]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = """ insert into soundCloud.soundCloud_app_history (session_number,item_in_session_number,artist_name,song_tittle,length)
        values (%s,%s,%s,%s,%s)"""
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query,(int(row[8]),int(row[3]),row[0],row[9],float(row[5])))

### Validate our Data Model using a SELECT

In [63]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
try:
  rows=  session.execute(""" select artist_name, song_tittle, length from soundCloud_app_history where session_number =338 and
  item_in_session_number =4""")
  for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(artist_name='Faithless', song_tittle='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [42]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command                   

In [64]:
# Task: Create a table for Query 2
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_session_songs (
        user_id INT,
        session_number INT,
        item_in_session_number INT,
        artist_name TEXT,
        song_title TEXT,
        fname TEXT,
        lname TEXT,
        PRIMARY KEY ((user_id, session_number), item_in_session_number)
    ) WITH CLUSTERING ORDER BY (item_in_session_number ASC);
    """)
    print("Table user_session_songs created.")
except Exception as e:
    print(e)


Table user_session_songs created.


### Let's insert our data into of table

In [80]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = """INSERT INTO user_session_songs (user_id, session_number, item_in_session_number, artist_name, song_title, fname, lname)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7
        session.execute(query, (int(row[10]), int(row[8]), int(row[3]), row[0], row[9], row[1], row[4]))

### Validate our Data Model using a SELECT

In [81]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
try:
   rows= session.execute("""
    select artist_name, song_title, fname,lname from user_session_songs
    where user_id=10 and session_number=182 order by item_in_session_number
    """)
   for row in rows:
    print(row)
except Exception as e:
    print(e)


Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", fname='Sylvie', lname='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', fname='Sylvie', lname='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', fname='Sylvie', lname='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', fname='Sylvie', lname='Cruz')


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [ ]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command                   

In [79]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS song_listeners (
        song_title TEXT,
        user_id INT,
        fname TEXT,
        lname TEXT,
        PRIMARY KEY (song_title, user_id)
    );
    """)
    print("Table song_listeners created.")
except Exception as e:
    print(e)

Table song_listeners created.


### Let's insert our data into of table

In [88]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        
        query = """ INSERT INTO song_listeners (song_title, user_id, fname, lname) VALUES (%s, %s, %s, %s)"""
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[9], int(row[10]), row[1], row[4]))


In [82]:
df.columns

Index(['artist_name', 'fname', 'gender', 'item_in_session_number', 'lname',
       'length', 'level', 'location', 'session_number', 'song_title',
       'user_id'],
      dtype='object')

### Validate our Data Model using a SELECT

In [92]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
try:
    rows= session.execute("""
    select fname, lname from song_listeners where song_title='All Hands Against His Own'
    """)
    for row in rows:
        print(row)
except Exception as e:
   print(e)

Row(fname='Jacqueline', lname='Lynch')
Row(fname='Tegan', lname='Levine')
Row(fname='Sara', lname='Johnson')


### Drop the tables before closing out the sessions

### Close the session and cluster connection¶